In [284]:
#important library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import warnings
import os
import time

In [285]:
print('Program is running.....')
print()
start_time=time.time()

Program is running.....



In [286]:
#Load the Dataset
a1=pd.read_excel("C:\\Users\\anike\\OneDrive\\Desktop\\Lang\\python\\Credit Risk Modeling\\case_study1.xlsx")
a2=pd.read_excel("C:\\Users\\anike\\OneDrive\\Desktop\\Lang\\python\\Credit Risk Modeling\\case_study2.xlsx")

In [287]:
df1=a1.copy()
df2=a2.copy()

In [288]:
df1.shape

(51336, 26)

In [289]:
df2.shape

(51336, 62)

In [290]:
df1=df1.loc[df1['Age_Oldest_TL']!=-99999]#just removes values and rows which has value -99999

In [291]:
df1.shape

(51296, 26)

In [292]:
#removing columns where values -99999 and it more than 10k times and where the value 
columns_to_be_removed=[]
for i in df2.columns:
  if df2.loc[df2[i]==-99999].shape[0]>10000:
    columns_to_be_removed.append(i)

In [293]:
df2.shape
df2=df2.drop(columns_to_be_removed,axis=1)

In [294]:
df2.shape

(51336, 54)

In [295]:
#removing columns now we will remove the rows 
for i in df2.columns:
  df2=df2.loc[df2[i]!=-99999]

In [296]:
df2.shape

(42066, 54)

In [297]:
df1.isna().sum()

PROSPECTID              0
Total_TL                0
Tot_Closed_TL           0
Tot_Active_TL           0
Total_TL_opened_L6M     0
Tot_TL_closed_L6M       0
pct_tl_open_L6M         0
pct_tl_closed_L6M       0
pct_active_tl           0
pct_closed_tl           0
Total_TL_opened_L12M    0
Tot_TL_closed_L12M      0
pct_tl_open_L12M        0
pct_tl_closed_L12M      0
Tot_Missed_Pmnt         0
Auto_TL                 0
CC_TL                   0
Consumer_TL             0
Gold_TL                 0
Home_TL                 0
PL_TL                   0
Secured_TL              0
Unsecured_TL            0
Other_TL                0
Age_Oldest_TL           0
Age_Newest_TL           0
dtype: int64

In [298]:
#Checking common 
for i in list(df1.columns):
  if i in list(df2.columns):
    print(i)

PROSPECTID


In [299]:
#merging the two df1 and df2
df=pd.merge(df1,df2,how='inner',left_on='PROSPECTID',right_on='PROSPECTID')

In [300]:
print(df)

       PROSPECTID  Total_TL  Tot_Closed_TL  Tot_Active_TL  \
0               1         5              4              1   
1               2         1              0              1   
2               3         8              0              8   
3               5         3              2              1   
4               6         6              5              1   
...           ...       ...            ...            ...   
42059       51332         3              0              3   
42060       51333         4              2              2   
42061       51334         2              1              1   
42062       51335         2              1              1   
42063       51336         1              0              1   

       Total_TL_opened_L6M  Tot_TL_closed_L6M  pct_tl_open_L6M  \
0                        0                  0            0.000   
1                        0                  0            0.000   
2                        1                  0            0.125   
3  

In [301]:
df.shape#merged datset

(42064, 79)

In [302]:
#divide features into categorical and numerical 
#1 categorical values/columns
for i in df.columns:
    if df[i].dtype=='object':
        print(i)

MARITALSTATUS
EDUCATION
GENDER
last_prod_enq2
first_prod_enq2
Approved_Flag


In [303]:
#2 numerical values
for i in df.columns:
    if df[i].dtype!='object':
        print(i)

PROSPECTID
Total_TL
Tot_Closed_TL
Tot_Active_TL
Total_TL_opened_L6M
Tot_TL_closed_L6M
pct_tl_open_L6M
pct_tl_closed_L6M
pct_active_tl
pct_closed_tl
Total_TL_opened_L12M
Tot_TL_closed_L12M
pct_tl_open_L12M
pct_tl_closed_L12M
Tot_Missed_Pmnt
Auto_TL
CC_TL
Consumer_TL
Gold_TL
Home_TL
PL_TL
Secured_TL
Unsecured_TL
Other_TL
Age_Oldest_TL
Age_Newest_TL
time_since_recent_payment
num_times_delinquent
max_recent_level_of_deliq
num_deliq_6mts
num_deliq_12mts
num_deliq_6_12mts
num_times_30p_dpd
num_times_60p_dpd
num_std
num_std_6mts
num_std_12mts
num_sub
num_sub_6mts
num_sub_12mts
num_dbt
num_dbt_6mts
num_dbt_12mts
num_lss
num_lss_6mts
num_lss_12mts
recent_level_of_deliq
tot_enq
CC_enq
CC_enq_L6m
CC_enq_L12m
PL_enq
PL_enq_L6m
PL_enq_L12m
time_since_recent_enq
enq_L12m
enq_L6m
enq_L3m
AGE
NETMONTHLYINCOME
Time_With_Curr_Empr
pct_of_active_TLs_ever
pct_opened_TLs_L6m_of_L12m
pct_currentBal_all_TL
CC_Flag
PL_Flag
pct_PL_enq_L6m_of_L12m
pct_CC_enq_L6m_of_L12m
pct_PL_enq_L6m_of_ever
pct_CC_enq_L6m_of_

In [304]:
#chi-square test 
#checking marital status and approved_flag are thse two associated
#h0-not associated-by default its true 
#h1-associated
#alpha(assumed .05)-Significance level- strictness level
# strong h1 is true oterwise h0 is true and h1 false 
#we are talikng alpha as 5% becuase we can affrod some mistakes here but there are domains and project were we take alpha 0.00001 where we cant afford any mistakes
#confidence-interval-5% rsk ci 100%-5% 0r 1-.05
#p-value is calulated using t-test,chisquare,anova and degree of freedom
#if p-value is less than aplha ho  is not accepted otherwise failed to reject h0--we do this we dont have correct knowledge thats why 
#chi-square test categorical vs categorical
#t-test cat vs numerica
#anova =cat vs numeric(>=2 categories)as categorical values are more than 2

In [305]:
#chi-square test between  categorical columns 
for i in ['MARITALSTATUS','EDUCATION','GENDER','last_prod_enq2','first_prod_enq2']:
    chi2,pval,_,_=chi2_contingency(pd.crosstab(df[i],df['Approved_Flag']))
    print(i,'---',pval)

MARITALSTATUS --- 3.578180861038862e-233
EDUCATION --- 2.6942265249737532e-30
GENDER --- 1.907936100186563e-05
last_prod_enq2 --- 0.0
first_prod_enq2 --- 7.84997610555419e-287


In [306]:
#since the values were pvalue is lesser than .05 is accepted


In [307]:
#numeric values/columns
numeric_columns=[]
for i in df.columns:
    if df[i].dtype!='object'and i not in['PROSPECTID','Approved_Flag']:
        numeric_columns.append(i)

In [308]:
print(numeric_columns)

['Total_TL', 'Tot_Closed_TL', 'Tot_Active_TL', 'Total_TL_opened_L6M', 'Tot_TL_closed_L6M', 'pct_tl_open_L6M', 'pct_tl_closed_L6M', 'pct_active_tl', 'pct_closed_tl', 'Total_TL_opened_L12M', 'Tot_TL_closed_L12M', 'pct_tl_open_L12M', 'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'Auto_TL', 'CC_TL', 'Consumer_TL', 'Gold_TL', 'Home_TL', 'PL_TL', 'Secured_TL', 'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL', 'Age_Newest_TL', 'time_since_recent_payment', 'num_times_delinquent', 'max_recent_level_of_deliq', 'num_deliq_6mts', 'num_deliq_12mts', 'num_deliq_6_12mts', 'num_times_30p_dpd', 'num_times_60p_dpd', 'num_std', 'num_std_6mts', 'num_std_12mts', 'num_sub', 'num_sub_6mts', 'num_sub_12mts', 'num_dbt', 'num_dbt_6mts', 'num_dbt_12mts', 'num_lss', 'num_lss_6mts', 'num_lss_12mts', 'recent_level_of_deliq', 'tot_enq', 'CC_enq', 'CC_enq_L6m', 'CC_enq_L12m', 'PL_enq', 'PL_enq_L6m', 'PL_enq_L12m', 'time_since_recent_enq', 'enq_L12m', 'enq_L6m', 'enq_L3m', 'AGE', 'NETMONTHLYINCOME', 'Time_With_Curr_Empr', 'pct_

In [309]:
#multicolinearity vs correlation
#Multicollinearity=Predictability of each features by other features
#correlation
#correlation for linear relation
#in convex functions,correlation gives misleading values
#what the plan is removing the columns which can be predicted by other using anova
#vif -variance inflation factor #used to indetify multicolanirity among features
#vif =1 no muticolinarity 1-5 small multicolinearity 5-10 moderate multicoli. above 10 high multicollinearity


In [310]:
vif_data=df[numeric_columns]
total_columns=vif_data.shape[1]
columns_to_be_kept=[]
column_index=0
for i in range (0,total_columns):
    vif_value=variance_inflation_factor(vif_data,column_index)
    print(column_index,'---',vif_value)
    if vif_value<=6:
        columns_to_be_kept.append(numeric_columns[i])
        column_index=column_index+1
    else:
        vif_data=vif_data.drop([numeric_columns[i]],axis=1)
    

c:\Users\anike\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf


c:\Users\anike\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf
0 --- 11.320180023967996
0 --- 8.363698035000336
0 --- 6.520647877790928
0 --- 5.149501618212625
1 --- 2.611111040579735


c:\Users\anike\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


2 --- inf
2 --- 1788.7926256209232
2 --- 8.601028256477228
2 --- 3.8328007921530785
3 --- 6.099653381646739
3 --- 5.581352009642762
4 --- 1.985584353098778


c:\Users\anike\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


5 --- inf
5 --- 4.809538302819343
6 --- 23.270628983464636
6 --- 30.595522588100053
6 --- 4.3843464059655854
7 --- 3.064658415523423
8 --- 2.898639771299253
9 --- 4.377876915347324
10 --- 2.207853583695844
11 --- 4.916914200506864
12 --- 5.214702030064725
13 --- 3.3861625024231476
14 --- 7.840583309478997
14 --- 5.255034641721438


c:\Users\anike\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


15 --- inf
15 --- 7.380634506427232
15 --- 1.4210050015175733
16 --- 8.083255010190316
16 --- 1.6241227524040112
17 --- 7.257811920140003
17 --- 15.59624383268298
17 --- 1.825857047132431
18 --- 1.5080839450032664
19 --- 2.172088834824577
20 --- 2.62339755352723
21 --- 2.2959970812106176
22 --- 7.360578319196439
22 --- 2.1602387773102554
23 --- 2.8686288267891458
24 --- 6.458218003637277
24 --- 2.8474118865638265
25 --- 4.753198156284083
26 --- 16.22735475594825
26 --- 6.424377256363877
26 --- 8.887080381808687
26 --- 2.3804746142952653
27 --- 8.609513476514548
27 --- 13.06755093547673
27 --- 3.500040056654654
28 --- 1.9087955874813773
29 --- 17.006562234161628
29 --- 10.730485153719197
29 --- 2.3538497522950275
30 --- 22.104855915136433
30 --- 2.7971639638512906
31 --- 3.4241712032176985
32 --- 10.175021454450935
32 --- 6.408710354561301
32 --- 1.001151196262561
33 --- 3.069197305397274
34 --- 2.8091261600643715
35 --- 20.249538381980678
35 --- 15.864576541593774
35 --- 1.833164974053

In [311]:
print(len(columns_to_be_kept))#there are 39 columns which are unique sequential 


39


In [312]:
#vif-parallel vs Sequential
from scipy.stats import f_oneway
columns_to_be_kept_numerical=[]
for i in columns_to_be_kept:
    a=list(df[i])
    b=list(df['Approved_Flag'])
    group_p1=[value for value,group in zip(a,b)if group=='P1']
    group_p2=[value for value,group in zip(a,b)if group=='P2']
    group_p3=[value for value,group in zip(a,b)if group=='P3']
    group_p4=[value for value,group in zip(a,b)if group=='P4']
    f_statistic,p_value=f_oneway(group_p1,group_p2,group_p3,group_p4)
    if p_value<=0.05:
        columns_to_be_kept_numerical.append(i)

    

In [313]:
print(len(columns_to_be_kept_numerical))

37


In [314]:
df.shape

(42064, 79)

In [315]:
df['EDUCATION'].unique()

array(['12TH', 'GRADUATE', 'SSC', 'POST-GRADUATE', 'UNDER GRADUATE',
       'OTHERS', 'PROFESSIONAL'], dtype=object)

In [316]:
features=columns_to_be_kept_numerical+['MARITALSTATUS','EDUCATION','GENDER','last_prod_enq2','first_prod_enq2']
df=df[features+['Approved_Flag']]

In [317]:
#we define some values to
#SSC-------->1
#12th-------->2
#GRADUATE--------->3
#UNDER GRADUATE---------->3
#POST-GRADUATE----------->4
#OTHERS---------->1
#PROFESSIONAL--------->3

In [318]:
df.loc[df['EDUCATION']=='SSC',['EDUCATION']]=1
df.loc[df['EDUCATION']=='12TH',['EDUCATION']]=2
df.loc[df['EDUCATION']=='GRADUATE',['EDUCATION']]=3
df.loc[df['EDUCATION']=='UNDER GRADUATE',['EDUCATION']]=3
df.loc[df['EDUCATION']=='POST-GRADUATE',['EDUCATION']]=4
df.loc[df['EDUCATION']=='OTHERS',['EDUCATION']]=1
df.loc[df['EDUCATION']=='PROFESSIONAL',['EDUCATION']]=3

In [319]:
df['EDUCATION'].value_counts()
df['EDUCATION']=df['EDUCATION'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pct_tl_open_L6M            42064 non-null  float64
 1   pct_tl_closed_L6M          42064 non-null  float64
 2   Tot_TL_closed_L12M         42064 non-null  int64  
 3   pct_tl_closed_L12M         42064 non-null  float64
 4   Tot_Missed_Pmnt            42064 non-null  int64  
 5   CC_TL                      42064 non-null  int64  
 6   Home_TL                    42064 non-null  int64  
 7   PL_TL                      42064 non-null  int64  
 8   Secured_TL                 42064 non-null  int64  
 9   Unsecured_TL               42064 non-null  int64  
 10  Other_TL                   42064 non-null  int64  
 11  Age_Oldest_TL              42064 non-null  int64  
 12  Age_Newest_TL              42064 non-null  int64  
 13  time_since_recent_payment  42064 non-null  int

In [320]:
df_encoded=pd.get_dummies(df,columns=['MARITALSTATUS','GENDER','last_prod_enq2','first_prod_enq2'])


In [321]:
#machine learning model fitting
#data preprocessing
#1.random Forest
y=df_encoded['Approved_Flag']
x=df_encoded.drop(['Approved_Flag'],axis=1)#added other features other than approved flag to x
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)#random_state can be anything 
rf_classifier=RandomForestClassifier(n_estimators=200,random_state=42)
rf_classifier.fit(x_train,y_train)
y_pred=rf_classifier.predict(x_test)
accuracy=accuracy_score(y_test,y_pred)
print()
print(f'Accuracy:{accuracy}')
print()
precision,recall,f1_score,_=precision_recall_fscore_support(y_test,y_pred)
for i ,v in enumerate(['p1','p2','p3','p4']):
    print(f"Class:{v}")
    print(f"Precision:{precision[i]}")
    print(f"Recall:{recall[i]}")
    print()


Accuracy:0.7636990372043266

Class:p1
Precision:0.8370457209847597
Recall:0.7041420118343196

Class:p2
Precision:0.7957519116397621
Recall:0.9282457879088206

Class:p3
Precision:0.4423380726698262
Recall:0.21132075471698114

Class:p4
Precision:0.7178502879078695
Recall:0.7269193391642371



In [322]:
#2.XGBOOST


In [323]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

xgb_classifier=xgb.XGBClassifier(objective='multi:softmax',num_class=4)
y=df_encoded['Approved_Flag']
x=df_encoded.drop(['Approved_Flag'],axis=1)
label_encoder=LabelEncoder()
y_encoded=label_encoder.fit_transform(y)
x_train,x_test,y_train,y_test=train_test_split(x,y_encoded,test_size=0.1,random_state=42)
xgb_classifier.fit(x_train,y_train)
y_pred=xgb_classifier.predict(x_test)
accuracy=accuracy_score(y_test,y_pred)
print()

print(f'Accuracy:{accuracy:.2f}')


Accuracy:0.78


In [326]:
import xgboost as xgb
# Define the hyperparameters grid
param_grid = {
    'max_depth': [3, 6, 9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'alpha'       : [1,10,100]
}
index=0
answers_grid= {
    'combination'      :[],
    'train_accuracy'   :[],
    'test_accuracy'    :[],
    'colsample_bytree' :[],
    'learning_rate'    :[],
    'max_depth'        :[],
    'alpha'            :[],
    'n_estimators'     :[]        
    
}
#Loop throung combination hyperparameter
for colsample_bytree in param_grid['colsample_bytree']:
    for learning_rate in param_grid['learning_rate']:
        for max_depth in param_grid['max_depth']:
            for alpha in param_grid['alpha']:
                for n_estimators in param_grid['n_estimators']:
                    index=index+1
                    #define and train XGBOOst model
                    model=xgb.XGBClassifier(objective='multi:softmax',
                                            num_class=4,
                                            colsample_bytree=colsample_bytree,
                                            learning_rate=learning_rate,
                                            max_depth=max_depth,
                                            alpha=alpha,
                                            n_estimators=n_estimators)
                    y=df_encoded['Approved_Flag']
                    x=df_encoded.drop(['Approved_Flag'],axis=1)
                    label_encoder=LabelEncoder()
                    y_encoded=label_encoder.fit_transform(y)
                    x_train,x_test,y_train,y_test=train_test_split(x,y_encoded,test_size=0.1,random_state=42)
                    model.fit(x_train,y_train)
                    y_pred_train=model.predict(x_train)
                    y_pred_test=model.predict(x_test)
                    
                    #calculate thr test and train result
                    train_accuracy=accuracy_score(y_train,y_pred_train)
                    test_accuracy=accuracy_score(y_test,y_pred_test)
                    
                    #include into the lists 
                    answers_grid['combination'].append(index)
                    answers_grid['train_accuracy'].append(train_accuracy)
                    answers_grid['test_accuracy'].append(test_accuracy)
                    answers_grid['colsample_bytree'].append(colsample_bytree)
                    answers_grid['learning_rate'].append(learning_rate)
                    answers_grid['max_depth'].append(max_depth)
                    answers_grid['alpha'].append(alpha)
                    answers_grid['n_estimators'].append(n_estimators)
                    #print results for this combination 
                    print(f"Combination{index}")
                    print(f"colsample_bytree:{colsample_bytree},learning_rate:{learning_rate}")
                    print(f"Train Accuracy:{train_accuracy:.2f}")
                    print(f"Test Accuracy:{test_accuracy:.2f}")
                    print("-"*30)
                    

                


Combination1
colsample_bytree:0.7,learning_rate:0.01
Train Accuracy:0.73
Test Accuracy:0.72
------------------------------
Combination2
colsample_bytree:0.7,learning_rate:0.01
Train Accuracy:0.74
Test Accuracy:0.73
------------------------------
Combination3
colsample_bytree:0.7,learning_rate:0.01
Train Accuracy:0.75
Test Accuracy:0.74
------------------------------
Combination4
colsample_bytree:0.7,learning_rate:0.01
Train Accuracy:0.73
Test Accuracy:0.72
------------------------------
Combination5
colsample_bytree:0.7,learning_rate:0.01
Train Accuracy:0.74
Test Accuracy:0.73
------------------------------
Combination6
colsample_bytree:0.7,learning_rate:0.01
Train Accuracy:0.74
Test Accuracy:0.74
------------------------------
Combination7
colsample_bytree:0.7,learning_rate:0.01
Train Accuracy:0.72
Test Accuracy:0.72
------------------------------
Combination8
colsample_bytree:0.7,learning_rate:0.01
Train Accuracy:0.73
Test Accuracy:0.72
------------------------------
Combination9
col

In [347]:
a3=pd.read_excel("C:\\Users\\anike\\OneDrive\\Desktop\\Lang\\python\\Credit Risk Modeling\\Unseen_Dataset.xlsx")
cols_in_df=list(df.columns)
cols_in_df.pop(42)
df_unseen=a3[cols_in_df]
df_unseen['MARITALSTATUS'].unique()
df_unseen['EDUCATION'].unique()
df_unseen['GENDER'].unique()
df_unseen['last_prod_enq2'].unique()
df_unseen['first_prod_enq2'].unique()
#education encoding as done above for seen data now we are doing for unseen data 
df_unseen.loc[df_unseen ['EDUCATION']=='SSC',['EDUCATION']]=1
df_unseen.loc[df_unseen ['EDUCATION']=='12TH',['EDUCATION']]=1
df_unseen.loc[df_unseen ['EDUCATION']=='GRADUATE',['EDUCATION']]=1
df_unseen.loc[df_unseen ['EDUCATION']=='UNDER GRADUATE',['EDUCATION']]=1
df_unseen.loc[df_unseen ['EDUCATION']=='POST-GRADUATE',['EDUCATION']]=1
df_unseen.loc[df_unseen ['EDUCATION']=='OTHERS',['EDUCATION']]=1
df_unseen.loc[df_unseen ['EDUCATION']=='PROFESSIONAL',['EDUCATION']]=1

df_unseen['EDUCATION'].value_counts()
df_unseen['EDUCATION']=df_unseen['EDUCATION'].astype(int)
df_unseen.info()

df_encoded_unseen=pd.get_dummies(df_unseen,columns=['MARITALSTATUS','GENDER','last_prod_enq2','first_prod_enq2'])
df_encoded_unseen.info()
k=df_encoded.describe()
#the below parametrs are set after checking all combiunation and which performed best 
model=xgb.XGBClassifier(objective='multi:softmax',
                        num_class=4,
                        colsample_bytree=0.9,
                        learning_rate=1,
                        max_depth=3,
                        alpha=10,
                        n_estimators=100)
#here we are prediction the approved flag for unseen data


model.fit(x_train,y_train)
y_pred_unseen=model.predict(df_encoded_unseen)

a3['Target_variable']=y_pred_unseen
a3.to_excel("C:\\Users\\anike\\OneDrive\\Desktop\\Lang\\python\\Credit Risk Modeling\\Final_Prediction.xlsx",index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 42 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pct_tl_open_L6M            100 non-null    float64
 1   pct_tl_closed_L6M          100 non-null    float64
 2   Tot_TL_closed_L12M         100 non-null    int64  
 3   pct_tl_closed_L12M         100 non-null    float64
 4   Tot_Missed_Pmnt            100 non-null    int64  
 5   CC_TL                      100 non-null    int64  
 6   Home_TL                    100 non-null    int64  
 7   PL_TL                      100 non-null    int64  
 8   Secured_TL                 100 non-null    int64  
 9   Unsecured_TL               100 non-null    int64  
 10  Other_TL                   100 non-null    int64  
 11  Age_Oldest_TL              100 non-null    int64  
 12  Age_Newest_TL              100 non-null    int64  
 13  time_since_recent_payment  100 non-null    int64  


In [344]:
import pickle
filename='eps_v1.sav'
pickle.dump(model,open(filename,'wb'))

In [346]:
load_model=pickle.load(open(filename,'rb'))
arg=x_train[:2] 
load_model.predict(arg)

array([1, 1])

In [ ]:
#3.decision tree
from sklearn.tree import DecisionTreeClassifier
y=df_encoded['Approved_Flag']
x=df_encoded.drop(['Approved_Flag'],axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
dt_model=DecisionTreeClassifier(max_depth=20,min_samples_split=10)
dt_model.fit(x_train,y_train)
y_pred=dt_model.predict(x_test)
accuracy=accuracy_score(y_test,y_pred)
print()
print(f"Accuracy:{accuracy:.2f}")



Accuracy:0.71


In [ ]:
#xgboost performed better than decison tree and random forest
#now we can start better feature engineering and scaling to make xgboost better
#accuracy--->out of total values,how many are correctly predicted
#Recall------>out of total values of class total correctly predicted
#precision----->out of total values predicted for a class,how many are correctly predicted
#f1-score------->2*p*r/(p+r)

In [ ]:
#4.logistic regression
from sklearn.linear_model import LogisticRegression
y=df_encoded['Approved_Flag']
x=df_encoded.drop(['Approved_Flag'],axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# Initialize the logistic regression classifier
logistic_regression = LogisticRegression(random_state=52)

# Fit the classifier to the training data
logistic_regression.fit(x_train, y_train)

# Make predictions on the testing data
y_pred = logistic_regression.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6087008201592773


c:\Users\anike\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#xgboost still performed better
#different matrix mae,mse,rmse,mape(not scale dependent)
#hypermeter tuning in xgboost
